In [17]:
# Parameters
n_contributions = 21
n_punishments = 31
n_cross_val = 2
fraction_training = 0.1
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
output_path = "../../data/training/dev"
labels = {}
model_name = "graph"
model_args = {
    "add_rnn": False,
    "add_edge_model": False,
    "add_global_model": False,
    "hidden_size": 10,
    "x_encoding": [
        {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
        {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
        {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
        {"name": "prev_common_good", "norm": 128, "etype": "float"},
        {"name": "prev_valid", "etype": "bool"},
    ],
    "u_encoding": [{"name": "prev_common_good", "norm": 128, "etype": "float"}],
}
optimizer_args = {"lr": 0.0001, "weight_decay": 1e-05}
train_args = {"epochs": 100, "batch_size": 20, "clamp_grad": 1, "eval_period": 10}
device = "cpu"

In [8]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

output_path = os.path.join(output_path, 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
df = pd.read_csv(data_file)


data = create_torch_data(df)
syn_data = create_syn_data(n_contribution=21, n_punishment=31)

In [19]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']

def create_fully_connected(n_nodes):
    return th.tensor([[i,j]
        for i in range(n_nodes)
        for j in range(n_nodes)
    ]).T

def encode(model, data, *, mask=True, index=False, x_encode=True, y_encode=True, u_encode=False, device, n_player=4):
    data = {
        'mask': data['valid'] if mask else None,
        'x': model.x_encoder(**data) if x_encode else None,
        'y_enc': model.y_encoder(**data) if y_encode else None,
        'y': data['contributions'] if y_encode else None,
        'u': model.u_encoder(**data) if u_encode and hasattr(model, 'u_encoder') else None,
        'info': th.stack([data[c] for c in syn_index], dim=-1) if index else None,
    }
    data = {
        k: v.to(device)
        for k, v in data.items()
        if v is not None
    }

    n_episodes, n_agents, n_rounds, _ = data['x'].shape

    edge_attr = th.zeros(n_player*n_player, n_rounds,0)
    edge_index = create_fully_connected(n_player)

    n_episodes = list(data.values())[0].shape[0]
    dataset = [
        Data(**{k: v[i] for k, v in data.items()}, edge_attr=edge_attr, edge_index=edge_index, idx=i, group_idx=i, num_nodes=n_player)
        for i in range(n_episodes)
    ]
    return dataset


for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val, fraction_training)):
    model = AH_MODELS[model_name](
        n_contributions=n_contributions, n_punishments=n_punishments,
        **model_args).to(th_device)

    train_data_ = encode(model, train_data, mask=True, u_encode=True, device=th_device)
    test_data_ = encode(model, test_data, mask=True, u_encode=True, device=th_device)
    syn_data_ = encode(model, syn_data, mask=False, y_encode=False, u_encode=True, index=True, device=th_device)

    syn_df = using_multiindex(
        Batch.from_data_list(syn_data_)['info'], ['idx', 'round_number'], syn_index)

    ev.set_data(test=test_data_, train=train_data_, syn=syn_data_, syn_df=syn_df)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):

            

            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            mask = batch_data['mask'].flatten()
            loss = loss_fn(py, y_true)
            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        if e % train_args['eval_period'] == 0:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)
            ev.eval_set(model, 'train')
            ev.eval_set(model, 'test')
            sum_loss = 0
            n_steps = 0
    ev.eval_sync(model, syn_index=syn_index)

ev.save(output_path, labels)
model_path = os.path.join(output_path, 'model.pt')
model.save(model_path)

[15, 20, 27, 32, 84, 102] [1, 2, 3, 4, 6, 7, 11, 13, 16, 17, 18, 19, 22, 24, 25, 26, 28, 29, 31, 35, 36, 37, 41, 42, 43, 44, 46, 47, 50, 53, 55, 56, 59, 60, 61, 62, 63, 70, 72, 77, 79, 80, 82, 87, 89, 90, 91, 92, 93, 94, 95, 101, 104, 105, 110, 111, 113, 114, 116, 117, 118, 119, 121, 123, 124, 125, 128, 131]
CV 0 | Epoch 0 | Loss 3.039447069168091
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 10 | Loss 3.0376778125762938
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 20 | Loss 3.034489154815674
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 30 | Loss 3.031338024139404
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 40 | Loss 3.028216338157654
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 50 | Loss 3.0251142978668213
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 60 | Loss 3.0220367908477783
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 70 | Loss 3.01897611618042
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 80 | Loss 3.0159239292144777
0
1
2
3
4
5
6
7
8
9
CV 0 | Epoch 90 | Loss 3.012874722480774
0
1
2
3
4
5
6
7
8
9
[2, 18, 37, 72, 101, 117] [0, 5, 8, 9, 10, 12, 14, 15, 20, 21, 23, 27, 30, 32,